In [ ]:
from azureml.core import Workspace, ScriptRunConfig, Datastore, Dataset
from azureml.core.compute import ComputeTarget

In [ ]:
ws= Workspace.from_config("./config")

In [ ]:
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [ ]:
from azureml.core import LinkedService, SynapseWorkspaceLinkedServiceConfiguration
synapse_link_config= SynapseWorkspaceLinkedServiceConfiguration(
    subscription_id="",
    resource_group="",
    name="synapse_name"   # we have to take synapse cluster name

)
linked_service = LinkedService.register(
    workspace=ws,
    name='synapse-link',
    linked_service_config=synapse_link_config
)

In [ ]:
from azureml.core.compute import SynapseCompute, ComputeTarget
spark_pool_name= ""     #we have to take spark pool compute name
attached_synapse_name= ""
attach_config = SynapseCompute.attach_configuration(
    linked_service,
    type="SynapseSparkPool",
    pool_name=spark_pool_name
)
synapse_compute= ComputeTarget.attach(
    workspace=ws,
    name=attached_synapse_name,
    attach_config=attach_config
)
synapse_compute.wait_for_completion(True)

In [ ]:
cluster= ws.compute_targets[attached_synapse_name]

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies   

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
from azureml.core.runconfig import RunConfiguration 
run_config = RunConfiguration()
run_config.target= cluster
run_config.environment = myenv 

In [ ]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_databricks_step")

In [ ]:
script_config = ScriptRunConfig(source_directory=".",
                                script="TrainingScript.py",
                                environment=myenv,
                                run_config=run_config) 
new_run1 = experiment.submit(config=script_config)
new_run1.wait_for_completion()

In [ ]:
from azureml.pipeline.steps import PythonScriptStep 
from azureml.pipeline.core import PipelineData 
input_ds= ws.datasets.get("Titanic_selected_columns")
datafolder= PipelineData('datafolder', datastore=ws.get_default_datastore())

In [ ]:
dataprep_step3= PythonScriptStep(name="Datapreparation",
                                source_directory="Training_python_scripts/",
                                script_name= "Dataprep_pipeline.py",
                                inputs=[input_ds.as_named_input('raw_data')],
                                outputs=[datafolder],
                                runconfig=run_config,
                                arguments=['--datafolder', datafolder])

In [ ]:
train_step3= PythonScriptStep(name="ModelTraining",
                             source_directory="Training_python_scripts/",
                             script_name="Training_pipeline_explain.py",
                             inputs=[datafolder],
                             runconfig=run_config,
                             arguments=['--datafolder',datafolder])

In [ ]:
steps= [dataprep_step3,train_step3]

In [ ]:
from azureml.pipeline.core import Pipeline 
new_pipeline = Pipeline(workspace=ws, steps=steps)

In [ ]:
new_experiment= Experiment(workspace=ws, name="pipelineexp01")
pipeline_exp_run= new_experiment.submit(new_pipeline)
pipeline_exp_run.wait_for_completion(show_output=True)